In [1]:
import os
os.chdir('../..')

In [2]:
from pipelines.util import *
import duckdb
import pandas as pd

In [3]:
# get the data from EDD
URL = 'https://github.com/economic-analytics/edd/raw/main/data/parquet/PROD_LAD.parquet'
con = duckdb.connect()
data = con.execute(f"SELECT * FROM '{URL}'").fetchdf()

In [4]:
data['dates.date'].unique()
data['unix'] = pd.to_datetime(data['dates.date'], format=f'%Y-%m-%d').astype(int).div(10**6).astype(int)
data['decimal_date'] = data['unix'].div((86400*365.25)).add(1970).round(2)

In [5]:
data = data[data['decimal_date'] == max(data['decimal_date'].unique())]

In [6]:
data = data.pivot(index=['geography.code', 'geography.name'], columns='variable.name', values='value')

In [7]:
data.to_csv('src/themes/innovation-change/productivity/_data/most_recent_productivity_local_authority.csv')